In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [2]:
import os
import json

directory = "data_korean_history"
documents = []

for root, _, files in os.walk(directory):
    for file in files:
        data_dir = os.path.join(root, file)

        with open(data_dir, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        text_data = data['query']['pages'][list(data['query']['pages'].keys())[0]]['extract'].split('\n\n\n')
        text_title = data['query']['pages'][list(data['query']['pages'].keys())[0]]['title']
        for t in text_data:
            if len(t) > 300:
                documents.append(text_title + '\n\n' + t)
        

    #with open('data/'+f, 'r', encoding='utf-8') as file:
        #content = file.read()
        #content = content.split('\n\n')
        #documents = documents + content


In [3]:
import faiss
import numpy as np

# Convert documents to embeddings
doc_embeddings = np.array(embedding_model.encode(documents), dtype=np.float32)

# Create a FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension) # Index - Vector Embedding Mapping
index.add(doc_embeddings)


In [4]:
def retrieve_similar_documents(query, top_k=5):
    query_embedding = np.array([embedding_model.encode(query)], dtype=np.float32)
    distances, indices = index.search(query_embedding, top_k)
    
    results = [documents[i] for i in indices[0]]
    return results

In [5]:
def context_query(query):
    context = "\n\n".join(retrieve_similar_documents(query, top_k=1))
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    return prompt

## RAG with Gemma3-1b

In [6]:
from transformers import pipeline
import torch
pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)

Device set to use cuda


In [8]:
# Chatbot Demonstration
messages = [[]]
query_chain = ""

while True:
    query = input(">>")
    
    query_chain += query + '\n\n\n'
    messages[0].append({
            "role": "user",
            "content": [{"type": "text", "text": context_query(query_chain)},]})
    
    output = pipe(messages, max_new_tokens=1000)
    
    query_chain += output[0][0]['generated_text'][-1]['content'] + '\n\n\n'
    messages[0].append({
        "role": "assistant",
        "content": [{"type": "text", "text": output[0][0]['generated_text'][-1]['content']},]})
    
    print(output[0][0]['generated_text'][-1]['content'])
    print()

>> tell me about the imjin war


Okay, here’s a breakdown of the Imjin War, based on the provided context:

**What was it?**

The Imjin War (1592-1598) was a major conflict between China, Japan, and Korea. It was the first regional war involving massive armies, utilizing modern weaponry by Japanese forces.

**Key Details:**

*   **Participants:**
    *   **China:** Primarily employed its regular army, with estimates of up to 80,000 soldiers.
    *   **Japan:** The Japanese army consisted of up to 200,000 men, employing modern military technology.
    *   **Korea:** Korean forces were estimated to be in the hundreds of thousands.
*   **Scale & Scope:** This was the only military conflict between China, Japan, and Korea during a specific period (1281-1894).
*   **Impact:** The war significantly challenged China’s position as the dominant military power in East Asia and demonstrated a willingness of Chinese support for its tributary states.

**The Significance:**

The Imjin War is considered a pivotal moment in Asian his

>> can you tell me the causes


Okay, let’s delve into the causes of the Imjin War. Here’s a breakdown of the key factors that led to the conflict:

**Primary Causes:**

*   **Shifting Power Dynamics in Korea:** Korea was a relatively weak and fragmented kingdom at the time. The rise of the powerful Japanese navy and their ambitions to secure trade routes through Korea created a significant threat to Korean sovereignty.
*   **Japanese Expansionism:** Japan, under the leadership of the ambitious Tokugawa Ieyasu, was aggressively expanding its influence in Asia. They saw Korea as a strategic base for further expansion and aimed to solidify their control over the Korean peninsula.
*   **Chinese Discomfort with Japanese Expansion:** The Ming Chinese government, under the rule of the Hongwu Emperor, viewed Japanese encroachment as a direct challenge to their territorial integrity and a threat to their influence in the region. They feared the possibility of Japan directly challenging Chinese dominance.
*   **Trade Disputes

KeyboardInterrupt: Interrupted by user

In [11]:
messages

[[{'role': 'user',
   'content': [{'type': 'text',
     'text': "Context: Imjin War\n\n== Aftermath and conclusion ==\n\nSome sources state that the war was a Joseon and Ming victory, while others regard it to be inconclusive.\nThe Japanese invasions were East Asia's first regional wars involving massed armies equipped with modern weapons. The conflict saw the regular employment of Japanese armies of up to 200,000 men, Ming Chinese armies of 80,000, and the ongoing deployment of local Korean forces numbering in the hundreds of thousands.\nThe invasions also stood as a challenge to the existing Chinese world order on two levels: the military, in which the war challenged Ming China's status as the supreme military power in East Asia, and the political, in which the war affirmed Chinese willingness to aid in the protection of its tributary states.\nThis was the only military conflict between China, Japan, and Korea in the period between 1281 (the second Mongol invasion of Japan) and 1894 